# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv

from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****_1cA
LANGSMITH_API_KEY=****7ac4
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime


@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## What is a Dynamic Prompt?
A dynamic prompt allows the agent's behavior, instructions, and permissions to change at inference time **without redefining or recompiling the agent**. The same single agent instance can serve multiple types of users, automatically adapting its system prompt depending on who is calling it and their specific context.

## Build a Dynamic Prompt

Utilize runtime context and middleware to generate a dynamic prompt.

**Dynamic Prompts allow the agent's behavior and permissions to change at inference time without redefining or recompiling the agent** — the same agent instance serves both customers and employees, with the prompt adapting automatically based on who is calling it and their context.

## Best Practices

1. **Use Middleware for Injection**
   Use the `@dynamic_prompt` decorator from `langchain.agents.middleware.types` to intercept the `ModelRequest` and modify the prompt dynamically right before it hits the LLM.

2. **Define a Strict Runtime Context**
   Use a `dataclass` or Pydantic model (e.g., `RuntimeContext`) to strictly type the context properties you expect at runtime (like `is_employee`, `user_id`, or `db_connection`).

3. **Register Middleware upon Agent Creation**
   When calling `create_agent`, pass your robust dynamic prompt middleware function into the `middleware=[...]` argument and register the `context_schema=RuntimeContext`.

4. **Conditional Prompt Formatting**
   Inside your dynamic prompt function, evaluate the state (e.g., `request.runtime.context.is_employee`) and conditionally construct or format strings (like `table_limits`) to inject into a base `SYSTEM_PROMPT_TEMPLATE`.

## Use Cases in this Notebook

### 1. Role-Based Access Control (RBAC)
The primary use case demonstrated is data access separation. 
- **Customer (Non-Employee)**: If `is_employee=False`, the dynamic prompt restricts the database tables the LLM is allowed to query (e.g., only `Album`, `Artist`, `Track`, etc. — blocking access to sensitive tables like `Customer` or `Invoice`). 
- **Employee**: If `is_employee=True`, the restriction is lifted, and the LLM can query the full database to answer administrative questions (e.g., finding out the most costly purchase by a specific customer).

### 2. Multi-tenant Environments
When building a SaaS application, a dynamic prompt can load specific tenant rules, custom instructions, or organizational knowledge constraints into the prompt strictly based on the authenticated user's session context, avoiding the need to deploy separate AI agents for each tenant.

### 3. State-Dependent Policies
If your application can enter different states (e.g., "read-only mode" vs "write mode"), passing the current application state via the `RuntimeContext` can dynamically add system prompt rules that prevent the LLM from executing destructive tool calls when under "read-only mode".



In [6]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [8]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

I can’t see customer purchases with the tables I’m allowed to use (Album, Artist, Genre, Playlist, PlaylistTrack, Track). To find the most costly purchase by Frank Harris, I’d need access to Customer, Invoice, and InvoiceLine.

Could you allow those tables, and confirm whether you mean:
- The single largest invoice (total amount) by Frank Harris, or
- The most expensive single item (invoice line) he bought?

If instead you meant “the most expensive track by an artist named Frank Harris,” I can check the catalog with the current access—just confirm that’s what you want.


In [9]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_S73kzKOIDWrBpvy0QmEo7qmR)
 Call ID: call_S73kzKOIDWrBpvy0QmEo7qmR
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_tQPK4lLbjqanqsU0fMDH23Dv)
 Call ID: call_tQPK4lLbjqanqsU0fMDH23Dv
  Args:
    query: PRAGMA table_info('Customer');
================================= Tool Message =================================
Name: execute_sql

[(0, 'CustomerId', 'INTEGER', 1, None, 1), (1, 'FirstName', 'NVARCHAR(40)', 1, None, 0), (2, 'LastNa